In [1]:
# A whole lot of imports
import rasterio
import rasterio.transform
import rasterio.mask
import fiona
import shapely
import shapely.geometry
from shapely.geometry import MultiPolygon, Polygon
import pyproj

from matplotlib import pyplot as plt
import numpy as np
import mpl_toolkits.basemap as pbm

import rasterio.features
import rasterio.windows
from rasterio.windows import Window

In [2]:
# Raw code for masking a raster using the rasterio.mask.mask() function
# No longer part of rasterio package so should just be called by mask()
# Added feature where parameter out_dtype can be set
# and is fed into the out_dtype parameter of the read() function for reading a file


"""Mask the area outside of the input shapes with no data."""

import logging
import warnings

import numpy as np

from rasterio.errors import WindowError
from rasterio.features import geometry_mask, geometry_window


logger = logging.getLogger(__name__)


def raster_geometry_mask(dataset, shapes, all_touched=False, invert=False,
                         crop=False, pad=False, pad_width=0.5):
    """Create a mask from shapes, transform, and optional window within original
    raster.
    By default, mask is intended for use as a numpy mask, where pixels that
    overlap shapes are False.
    If shapes do not overlap the raster and crop=True, a ValueError is
    raised.  Otherwise, a warning is raised, and a completely True mask
    is returned (if invert is False).
    Parameters
    ----------
    dataset : a dataset object opened in 'r' mode
        Raster for which the mask will be created.
    shapes : iterable object
        The values must be a GeoJSON-like dict or an object that implements
        the Python geo interface protocol (such as a Shapely Polygon).
    all_touched : bool (opt)
        Include a pixel in the mask if it touches any of the shapes.
        If False (default), include a pixel only if its center is within one of
        the shapes, or if it is selected by Bresenham's line algorithm.
    invert : bool (opt)
        If False (default), mask will be `False` inside shapes and `True`
        outside.  If True, mask will be `True` inside shapes and `False`
        outside.
    crop : bool (opt)
        Whether to crop the dataset to the extent of the shapes. Defaults to
        False.
    pad : bool (opt)
        If True, the features will be padded in each direction by
        one half of a pixel prior to cropping dataset. Defaults to False.
    pad_width : float (opt)
        If pad is set (to maintain back-compatibility), then this will be the
        pixel-size width of the padding around the mask.
    Returns
    -------
    tuple
        Three elements:
            mask : numpy ndarray of type 'bool'
                Mask that is `True` outside shapes, and `False` within shapes.
            out_transform : affine.Affine()
                Information for mapping pixel coordinates in `masked` to another
                coordinate system.
            window: rasterio.windows.Window instance
                Window within original raster covered by shapes.  None if crop
                is False.
    """
    if crop and invert:
        raise ValueError("crop and invert cannot both be True.")

    if crop and pad:
        pad_x = pad_width
        pad_y = pad_width
    else:
        pad_x = 0
        pad_y = 0

    north_up = dataset.transform.e <= 0
    rotated = dataset.transform.b != 0 or dataset.transform.d != 0

    try:
        window = geometry_window(dataset, shapes, north_up=north_up, rotated=rotated,
                                 pad_x=pad_x, pad_y=pad_y)

    except WindowError:
        # If shapes do not overlap raster, raise Exception or UserWarning
        # depending on value of crop
        if crop:
            raise ValueError('Input shapes do not overlap raster.')
        else:
            warnings.warn('shapes are outside bounds of raster. '
                          'Are they in different coordinate reference systems?')

        # Return an entirely True mask (if invert is False)
        mask = np.ones(shape=dataset.shape[-2:], dtype='bool') * (not invert)
        return mask, dataset.transform, None

    if crop:
        transform = dataset.window_transform(window)
        out_shape = (int(window.height), int(window.width))

    else:
        window = None
        transform = dataset.transform
        out_shape = (int(dataset.height), int(dataset.width))

    mask = geometry_mask(shapes, transform=transform, invert=invert,
                         out_shape=out_shape, all_touched=all_touched)

    return mask, transform, window


def mask(dataset, shapes, all_touched=False, invert=False, nodata=None,
         filled=True, crop=False, pad=False, pad_width=0.5, indexes=None, out_dtype = None):
    """Creates a masked or filled array using input shapes.
    Pixels are masked or set to nodata outside the input shapes, unless
    `invert` is `True`.
    If shapes do not overlap the raster and crop=True, a ValueError is
    raised.  Otherwise, a warning is raised.
    Parameters
    ----------
    dataset : a dataset object opened in 'r' mode
        Raster to which the mask will be applied.
    shapes : iterable object
        The values must be a GeoJSON-like dict or an object that implements
        the Python geo interface protocol (such as a Shapely Polygon).
    all_touched : bool (opt)
        Include a pixel in the mask if it touches any of the shapes.
        If False (default), include a pixel only if its center is within one of
        the shapes, or if it is selected by Bresenham's line algorithm.
    invert : bool (opt)
        If False (default) pixels outside shapes will be masked.  If True,
        pixels inside shape will be masked.
    nodata : int or float (opt)
        Value representing nodata within each raster band. If not set,
        defaults to the nodata value for the input raster. If there is no
        set nodata value for the raster, it defaults to 0.
    filled : bool (opt)
        If True, the pixels outside the features will be set to nodata.
        If False, the output array will contain the original pixel data,
        and only the mask will be based on shapes.  Defaults to True.
    crop : bool (opt)
        Whether to crop the raster to the extent of the shapes. Defaults to
        False.
    pad : bool (opt)
        If True, the features will be padded in each direction by
        one half of a pixel prior to cropping raster. Defaults to False.
    indexes : list of ints or a single int (opt)
        If `indexes` is a list, the result is a 3D array, but is
        a 2D array if it is a band index number.
    Returns
    -------
    tuple
        Two elements:
            masked : numpy ndarray or numpy.ma.MaskedArray
                Data contained in the raster after applying the mask. If
                `filled` is `True` and `invert` is `False`, the return will be
                an array where pixels outside shapes are set to the nodata value
                (or nodata inside shapes if `invert` is `True`).
                If `filled` is `False`, the return will be a MaskedArray in
                which pixels outside shapes are `True` (or `False` if `invert`
                is `True`).
            out_transform : affine.Affine()
                Information for mapping pixel coordinates in `masked` to another
                coordinate system.
    """

    if nodata is None:
        if dataset.nodata is not None:
            nodata = dataset.nodata
        else:
            nodata = 0

    shape_mask, transform, window = raster_geometry_mask(
        dataset, shapes, all_touched=all_touched, invert=invert, crop=crop,
        pad=pad, pad_width=pad_width)

    if indexes is None:
        out_shape = (dataset.count, ) + shape_mask.shape
    elif isinstance(indexes, int):
        out_shape = shape_mask.shape
    else:
        out_shape = (len(indexes), ) + shape_mask.shape

    out_image = dataset.read(
        window=window, out_shape=out_shape, masked=True, indexes=indexes, out_dtype = out_dtype)

    out_image.mask = out_image.mask | shape_mask

    if filled:
        out_image = out_image.filled(nodata)

    return out_image, transform

In [3]:
# Used to warp from one coordinate system to another
# warping crs
def warp_xy(x, y, old_crs, new_crs):
    """Warps a set of points from old_crs to new_crs."""
    if old_crs == new_crs:
        return x,y

    old_crs_proj = pyproj.Proj(old_crs)
    new_crs_proj = pyproj.Proj(new_crs)
    return pyproj.transform(old_crs_proj, new_crs_proj, x,y)

def warp_shapely(shp, old_crs, new_crs):
    """Uses proj to reproject shapes, NOT IN PLACE"""
    if old_crs['init'] == new_crs['init']:
        return shp

    old_crs_proj = pyproj.Proj(old_crs)
    new_crs_proj = pyproj.Proj(new_crs)
    return shapely.ops.transform(lambda x,y:pyproj.transform(old_crs_proj, new_crs_proj, x,y), shp)

def warp_shape(feature, old_crs, new_crs):
    """Uses proj to reproject shapes, IN PLACE"""
    if old_crs == new_crs:
        return
    if len(feature['geometry']['coordinates']) is 0:
        return

    # find the dimension -- can't trust the shape
    dim = -1
    ptr = feature['geometry']['coordinates']
    done = False
    while not done:
        if hasattr(ptr, '__len__'):        
            assert(len(ptr) is not 0)
            dim += 1
            ptr = ptr[0]
        else:
            done = True

    if dim == 0:
        # point
        x,y = warp_xy(np.array([feature['geometry']['coordinates'][0],]), np.array([feature['geometry']['coordinates'][1],]), old_crs, new_crs)
        feature['geometry']['coordinates'][0] = x[0]
        feature['geometry']['coordinates'][1] = x[1]
    elif dim == 1:
        # line-like or polygon with no holes
        coords = np.array(feature['geometry']['coordinates'],'d')
        assert(len(coords.shape) is 2 and coords.shape[1] in [2,3] )
        x,y = warp_xy(coords[:,0], coords[:,1], old_crs, new_crs)
        new_coords = [xy for xy in zip(x,y)]
        feature['geometry']['coordinates'] = new_coords
    elif dim == 2:
        # multi-line or polygon with holes
        for i in range(len(feature['geometry']['coordinates'])):
            coords = np.array(feature['geometry']['coordinates'][i],'d')
            assert(len(coords.shape) is 2 and coords.shape[1] in [2,3])
            x,y = warp_xy(coords[:,0], coords[:,1], old_crs, new_crs)
            new_coords = [xy for xy in zip(x,y)]
            feature['geometry']['coordinates'][i] = new_coords
    elif dim == 3:
        # multi-polygon
        for i in range(len(feature['geometry']['coordinates'])):
            for j in range(len(feature['geometry']['coordinates'][i])):
                coords = np.array(feature['geometry']['coordinates'][i][j],'d')
                assert(len(coords.shape) is 2 and coords.shape[1] in [2,3])
                x,y = warp_xy(coords[:,0], coords[:,1], old_crs, new_crs)
                new_coords = [xy for xy in zip(x,y)]
                feature['geometry']['coordinates'][i][j] = new_coords

In [4]:
# importing all the data that we will be using

# import all U.S. CBSA shapes
with fiona.open(r'..\Data (Used)\tl_2017_us_cbsa\tl_2017_us_cbsa.shp', 'r') as fid:
    cbsas = list(fid)
    cbsas_profile = fid.profile

# import the impervious surface raster
with rasterio.open(r'..\Data (Used)\NLCD_2016_Impervious_L48_20190405\NLCD_2016_Impervious_L48_20190405.img','r') as fid:
    rprof = fid.profile

In [5]:
# Provides info on our imported data
print("We found {} CBSA shapes".format(len(cbsas)))
print('What CRS are we working in?')
print('  crs of CBSAs:', cbsas_profile['crs'])
print('  crs of NLCD:', rprof['crs'])

We found 945 CBSA shapes
What CRS are we working in?
  crs of CBSAs: {'init': 'epsg:4269'}
  crs of NLCD: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,-0,-0,-0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1]]


In [6]:
# Prepares our data to be worked with


# changed coordinate system from the native crs of the cbsa shapes in the shapefile to the native crs of the raster
for shp in cbsas:
    warp_shape(shp, cbsas_profile['crs'], rprof['crs'])


# Defines the transform so that MSA boxes have accurate corners
transform = rprof['transform']

# Finds which cbsas are major and classify as MSAs
# Also omits msas from Puerto Rico, Hawaii, and Alaska
# msas is a list that contains the shapes for each MSA in the lower 48 states
msas = []
for cbsa in cbsas:
    cbsa_type = cbsa['properties']['LSAD']
    if cbsa_type == 'M1':
        if 'PR' not in cbsa['properties']['NAME'] and 'HI' not in cbsa['properties']['NAME'] and 'AK' not in cbsa['properties']['NAME']:
            msas.append(cbsa)
print("We are using " + str(len(msas)) + " MSAs")

# sets up shapely objects for each MSA
msas_shapely = []
for msa in msas:
    msas_shapely.append(shapely.geometry.shape(msa['geometry']))


# sets up a shorter array of msas to run tests with
# consists of first 7 Polygons and first 3 MultiPolygons
test_msas = []
for i in range(7):
    test_msas.append(msas[i])
test_msas.append(msas[13])
test_msas.append(msas[31])
test_msas.append(msas[121])

test_msas_shapely = []
for i in range(7):
    test_msas_shapely.append(msas_shapely[i])
test_msas_shapely.append(msas_shapely[13])
test_msas_shapely.append(msas_shapely[31])
test_msas_shapely.append(msas_shapely[121])

We are using 378 MSAs


In [7]:
# Can be used to search for an MSA by name
named_msas = [msa for msa in msas if 'Philadelphia' in msa['properties']['NAME']]
assert(len(named_msas) == 1)
named_msa = named_msas[0]
print([msa['properties']['NAME'] for msa in named_msas])

['Philadelphia-Camden-Wilmington, PA-NJ-DE-MD']


In [8]:
# Takes in an MSA and data raster and returns a box of the data from the raster
def box_msa(msa_shapely, path):
    msa_bounds = msa_shapely.bounds
    ll = msa_bounds[0], msa_bounds[1]
    ur = msa_bounds[2], msa_bounds[3]
    ij_ll = tuple(reversed(rasterio.transform.rowcol(transform, *ll)))
    ij_ur = tuple(reversed(rasterio.transform.rowcol(transform, *ur)))
    with rasterio.open(path,'r') as fid:
        msa_boxed_imp_surf = fid.read(1, window = ((ij_ur[1], ij_ll[1]), (ij_ll[0], ij_ur[0])))
    return boxed_msa, msa_bounds

In [9]:
# Returns raster data masked and cropped to an MSA
def masked_on_msa(msa, path):
    with rasterio.open(path,'r') as fid:
        array, transform = mask(fid, msa, crop=True, nodata = np.nan, out_dtype = rasterio.float32)
    return array[0,:,:], transform

In [10]:
# Takes in an MSA and raster file path and plots them.
def plot_msa(msa, path, msa_shapely):
    if msa['geometry']['type'] == 'Polygon':
        x,y = msa['shape'].exterior.xy
        plt.plot(x,y)
    else:
        for poly in msa['shape']:
            x,y = poly.exterior.xy
            plt.plot(x,y)
    
    boxed_msa, msa_bounds = box_msa(msa_shapely, path)
    plt.imshow(boxed_msa, extent=[msa_bounds[0], msa_bounds[2], msa_bounds[1], msa_bounds[3]]) 
    plt.show()

In [11]:
with rasterio.open(r'..\Data (Used)\NLCD_2016_Impervious_L48_20190405\NLCD_2016_Impervious_L48_20190405.img','r') as fid:
    for i in range(len(test_msas)):
        plot_msa(test_msas[i], fid, test_msas_shapely[i])

KeyError: 'shape'

In [ ]:
print(r'..\Data (Used)\NLCD_2016_Impervious_L48_20190405\NLCD_2016_Impervious_L48_20190405.img')